In [ ]:
!pip install googletrans==3.1.0a0
!pip install -q datasets loralib sentencepiece
!pip uninstall transformers
!pip install -q git+https://github.com/zphang/transformers@c3dc391
!pip -q install git+https://github.com/huggingface/peft.git
!pip -q install bitsandbytes
!pip install gradio
#!pip install git+https://github.com/sanchit-gandhi/whisper-jax.git


In [ ]:
from tqdm.auto import tqdm
import librosa
from googletrans import Translator
from transformers import WhisperProcessor, WhisperForConditionalGeneration
from peft import PeftModel
from transformers import LLaMATokenizer, LLaMAForCausalLM, GenerationConfig
import textwrap



In [ ]:
from transformers import WhisperProcessor, WhisperForConditionalGeneration
#from datasets import load_dataset

# load model and processor
processor = WhisperProcessor.from_pretrained("openai/whisper-medium")
audio_model = WhisperForConditionalGeneration.from_pretrained("openai/whisper-medium")



In [ ]:
tokenizer = LLaMATokenizer.from_pretrained("decapoda-research/llama-7b-hf")

model = LLaMAForCausalLM.from_pretrained("decapoda-research/llama-7b-hf",load_in_8bit=True,device_map="auto",)
model = PeftModel.from_pretrained(model, "samwit/alpaca7B-lora")

In [ ]:
import librosa
file_path = 'test.mp3'
audio_data, sample_rate = librosa.load(file_path)
# Target sample rate (16000 Hz in this case)
target_sample_rate = 16000
# Resample the audio data to the target sample rate
audio_data_resampled = librosa.resample(audio_data, orig_sr=sample_rate, target_sr=target_sample_rate)


In [ ]:
from tqdm.auto import tqdm
batch_length_sec = 30
sample_rate = 16000
batch_length_samples = batch_length_sec * sample_rate

L =[]

audio_batches = [audio_data_resampled[i:i+batch_length_samples] for i in range(0, len(audio_data_resampled), batch_length_samples)]
for batch in tqdm(audio_batches):
    input_features = processor(batch, sampling_rate=target_sample_rate, return_tensors="pt").input_features

    # generate token ids
    predicted_ids = audio_model.generate(input_features)
    # decode token ids to text
    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=False)

    transcription = processor.batch_decode(predicted_ids, skip_special_tokens=True)

    L.append(transcription)


L= [b[0] for b in L]

In [ ]:
text= " ".join(L)

In [ ]:
translator = Translator()

In [ ]:
text = translator.translate(text, dest='en')
text = text.text
with open('test.txt', 'w') as f:
    f.write(text)

In [ ]:
def chat(text):
    inputs = tokenizer(
        text,
        return_tensors="pt",
    )
    input_ids = inputs["input_ids"].cuda()

    generation_config = GenerationConfig(
        temperature=0.1,
        top_p=0.95,
        repetition_penalty=1.2,
    )
    print("Generating...")
    generation_output = model.generate(
        input_ids=input_ids,
        generation_config=generation_config,
        return_dict_in_generate=True,
        output_scores=True,
        max_new_tokens=256,
    )
    for s in generation_output.sequences:
        a = tokenizer.decode(s)

    return tokenizer.decode(s)

In [ ]:
def chatBot(question):

  ques = translator.translate(question, dest='en')
  question = ques.text
  with open('test.txt') as f:
    contents = f.read()


  con = translator.translate(contents, dest='en')
  contents = con.text


  input_text =f'''Below is an instruction that describes a task. Write a response that appropriately completes the request.

  ### Instruction:
    {question} based on this text : {contents}.
  ### Response:
  '''

  L = chat(input_text)
  r = L.split("### Response:")[-1][1:]

  return r

In [ ]:
type(L.split("### Response:")[-1][1:])

In [ ]:
import gradio as gr
import random
import time

# Function to save the transcribed text into a text file
def save_to_file(transcription, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(transcription)

# transcription code
def get_text(url):
    # speech-to-text transcription here
    # ....
    transcription = "This is the speech-to-text transcription of the YouTube video from the URL: " + url

    # Save the transcription to a text file
    file_path = "transcription.txt"
    save_to_file(transcription, file_path)

    return "Transcription saved to file: " + file_path

# Function to save the transcribed text into a text file
def save_to_file(transcription, file_path):
    with open(file_path, 'w', encoding='utf-8') as file:
        file.write(transcription)

# Chatbot Interface
def response(message, history):
    L = chatBot(message)
    L = translator.translate(L, dest='ar')
    # For demonstration purposes, let's just return a random response from a list
    return L.text


with gr.Blocks() as demo:
  with gr.Row():
    with gr.Column(scale=1):
          input=gr.Textbox(placeholder='Youtube video URL')
          output=gr.Textbox(placeholder='Status', label='Status')
          button=gr.Button(value="Submit")
          button.click(fn=get_text, inputs=input, outputs=output)


    with gr.Column(scale=2):
      chatbot = gr.Chatbot()
      msg = gr.Textbox()
      clear = gr.ClearButton([msg, chatbot])

      def respond(message, chat_history):
          bot_message = chatBot(message)
          bot_message = translator.translate(bot_message, dest='ar')
          bot_message = bot_message.text
          #message#random.choice(["How are you?", "I'm fine", "I'm very hungry"])
          chat_history.append((message, bot_message))
          time.sleep(2)
          return "", chat_history

      def response(message, history):
        L = chatBot(message)
        L = translator.translate(L, dest='ar')
        return L.text


    msg.submit(respond, [msg, chatbot], [msg, chatbot])

# Launch the grouped blocks
demo.launch()
